## Import and Configure Everything We Need

In [7]:
%matplotlib inline

from collections import defaultdict as ddict, OrderedDict as odict
from typing import Any, Dict, List

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import xgboost as xgb
from rdkit.Chem import PandasTools, AllChem as Chem, Descriptors
from rdkit.ML.Descriptors.MoleculeDescriptors import MolecularDescriptorCalculator
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, r2_score, mean_squared_error
from sklearn.model_selection import KFold
from sklearn.neural_network import MLPRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
import deepchem as dc
import torch

from rdkit import RDLogger

RDLogger.DisableLog('rdApp.*') 

import basic as b
import chemprop_ish as c

pd.set_option('display.float_format', lambda x: '%.3f' % x)  # Display floats without scientific notation

# In many cases NaN
not_used_desc = ['MaxPartialCharge', 'MinPartialCharge', 'MaxAbsPartialCharge', 'MinAbsPartialCharge']

# Create a descriptor calculator for all RDKit descriptors except the ones above
desc_calc = MolecularDescriptorCalculator([x for x in [x[0] for x in Descriptors.descList] if x not in not_used_desc])

---
## Loading Precombined Dataset

In [8]:
data = pd.read_csv('combisolv_exp2.csv')
solute = data['smiles_solute'].tolist()
solvent = data['smiles_solvent'].tolist()
pka = data['dGsolv_avg [kcal/mol]'].tolist()
sol_solv = [[x,y] for x,y in zip(solute,solvent)]
#preprocess pka too

In [9]:
H_list = []
for x in range(len(sol_solv)):
    if sol_solv[x][0] in ["[H][H]","[2H][2H]","[HH]"]:
        H_list.append(x)
for x in sorted(H_list, reverse = True):
    del sol_solv[x]
    del pka[x]

---
## Training torch models
#### Using the following training sets with 5-fold cross-validation (shuffled)
1. Solute / solvent pairs

---
### Torch models

In [32]:
seed = 24
verbose = False

y_data = torch.Tensor(pka)
x_data = sol_solv
models = ddict(odict)

In [39]:
print(models[name].cv_scores.items())

dict_items([('mean_absolute_error', [5.5441566, 5.816946, 5.651475, 5.565803, 5.787024]), ('rmse', [7.0528293, 7.3005733, 7.131489, 7.035141, 7.3208847]), ('r2_score', [-1.4606123327346294, -1.5894076810621427, -1.453483740742597, -1.5079131981865617, -1.534027946198456])])


In [41]:
def generate_score_board(name):
    print(f'{name} CV Scores:')
    for k, v in models[name].cv_scores.items():
         print(f'\t\t- {k}: {np.mean(v):.3f} ± {np.std(v):.3f}')
generate_score_board(name)

MPNN, no interaction CV Scores:
		- mean_absolute_error: 5.673 ± 0.112
		- rmse: 7.168 ± 0.121
		- r2_score: -1.509 ± 0.050


In [33]:
args = c.TrainArgs()
args.__dict__.update({"depth":3, "dropout":0.2})
est_cls = c.double_MPNN(args)
name = 'MPNN, no interaction'

models[name] = b.train_cv_model(est_cls, x_data, y_data, params=None, random_state=seed)
generate_score_board(name)

/Users/u6676643/opt/anaconda3/lib/python3.8/site-packages/torch/nn/modules/loss.py:446: UserWarning: Using a target size (torch.Size([5])) that is different to the input size (torch.Size([5, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Fold done


/Users/u6676643/opt/anaconda3/lib/python3.8/site-packages/torch/nn/modules/loss.py:446: UserWarning: Using a target size (torch.Size([5])) that is different to the input size (torch.Size([5, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Fold done


/Users/u6676643/opt/anaconda3/lib/python3.8/site-packages/torch/nn/modules/loss.py:446: UserWarning: Using a target size (torch.Size([5])) that is different to the input size (torch.Size([5, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


...

/Users/u6676643/opt/anaconda3/lib/python3.8/site-packages/torch/nn/modules/loss.py:446: UserWarning: Using a target size (torch.Size([5])) that is different to the input size (torch.Size([5, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Fold done


/Users/u6676643/opt/anaconda3/lib/python3.8/site-packages/torch/nn/modules/loss.py:446: UserWarning: Using a target size (torch.Size([5])) that is different to the input size (torch.Size([5, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Fold done
MPNN, no interaction CV Scores:


AttributeError: 'CV_torch' object has no attribute 'items'

In [42]:
import imp
imp.reload(b)

<module 'basic' from '/Users/u6676643/codes/diympnn/deepchemMPNN/basic.py'>

In [43]:
args = c.TrainArgs()
args.__dict__.update({"depth":3, "dropout":0.2, "interaction":True})
est_cls = c.double_MPNN(args)
name = 'MPNN, interaction'

models[name] = b.train_cv_model(est_cls, x_data, y_data, params=None, random_state=seed)
generate_score_board(name)

/Users/u6676643/opt/anaconda3/lib/python3.8/site-packages/torch/nn/modules/loss.py:446: UserWarning: Using a target size (torch.Size([5])) that is different to the input size (torch.Size([5, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Fold done


/Users/u6676643/opt/anaconda3/lib/python3.8/site-packages/torch/nn/modules/loss.py:446: UserWarning: Using a target size (torch.Size([5])) that is different to the input size (torch.Size([5, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Fold done


/Users/u6676643/opt/anaconda3/lib/python3.8/site-packages/torch/nn/modules/loss.py:446: UserWarning: Using a target size (torch.Size([5])) that is different to the input size (torch.Size([5, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Fold done


/Users/u6676643/opt/anaconda3/lib/python3.8/site-packages/torch/nn/modules/loss.py:446: UserWarning: Using a target size (torch.Size([5])) that is different to the input size (torch.Size([5, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Fold done


/Users/u6676643/opt/anaconda3/lib/python3.8/site-packages/torch/nn/modules/loss.py:446: UserWarning: Using a target size (torch.Size([5])) that is different to the input size (torch.Size([5, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Fold done
MPNN, interaction CV Scores:
		- mean_absolute_error: 5.673 ± 0.112
		- rmse: 7.168 ± 0.121
		- r2_score: -1.509 ± 0.050


In [44]:
args = c.TrainArgs()
args.__dict__.update({"depth":3, "dropout":0.2, "interaction":True, "atom_messages":False})
est_cls = c.double_MPNN(args)
name = 'D-MPNN, interaction'

models[name] = b.train_cv_model(est_cls, x_data, y_data, params=None, random_state=seed)
generate_score_board(name)

/Users/u6676643/opt/anaconda3/lib/python3.8/site-packages/torch/nn/modules/loss.py:446: UserWarning: Using a target size (torch.Size([5])) that is different to the input size (torch.Size([5, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Fold done


/Users/u6676643/opt/anaconda3/lib/python3.8/site-packages/torch/nn/modules/loss.py:446: UserWarning: Using a target size (torch.Size([5])) that is different to the input size (torch.Size([5, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Fold done


/Users/u6676643/opt/anaconda3/lib/python3.8/site-packages/torch/nn/modules/loss.py:446: UserWarning: Using a target size (torch.Size([5])) that is different to the input size (torch.Size([5, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Fold done


/Users/u6676643/opt/anaconda3/lib/python3.8/site-packages/torch/nn/modules/loss.py:446: UserWarning: Using a target size (torch.Size([5])) that is different to the input size (torch.Size([5, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Fold done


/Users/u6676643/opt/anaconda3/lib/python3.8/site-packages/torch/nn/modules/loss.py:446: UserWarning: Using a target size (torch.Size([5])) that is different to the input size (torch.Size([5, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Fold done
D-MPNN, interaction CV Scores:
		- mean_absolute_error: 5.673 ± 0.112
		- rmse: 7.168 ± 0.121
		- r2_score: -1.509 ± 0.050


In [45]:
args = c.TrainArgs()
args.__dict__.update({"depth":3, "dropout":0.2, "interaction":False, "atom_messages":False})
est_cls = c.double_MPNN(args)
name = 'D-MPNN, no interaction'

models[name] = b.train_cv_model(est_cls, x_data, y_data, params=None, random_state=seed)
generate_score_board(name)

/Users/u6676643/opt/anaconda3/lib/python3.8/site-packages/torch/nn/modules/loss.py:446: UserWarning: Using a target size (torch.Size([5])) that is different to the input size (torch.Size([5, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Fold done


/Users/u6676643/opt/anaconda3/lib/python3.8/site-packages/torch/nn/modules/loss.py:446: UserWarning: Using a target size (torch.Size([5])) that is different to the input size (torch.Size([5, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Fold done


/Users/u6676643/opt/anaconda3/lib/python3.8/site-packages/torch/nn/modules/loss.py:446: UserWarning: Using a target size (torch.Size([5])) that is different to the input size (torch.Size([5, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Fold done


/Users/u6676643/opt/anaconda3/lib/python3.8/site-packages/torch/nn/modules/loss.py:446: UserWarning: Using a target size (torch.Size([5])) that is different to the input size (torch.Size([5, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Fold done


/Users/u6676643/opt/anaconda3/lib/python3.8/site-packages/torch/nn/modules/loss.py:446: UserWarning: Using a target size (torch.Size([5])) that is different to the input size (torch.Size([5, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Fold done
D-MPNN, no interaction CV Scores:
		- mean_absolute_error: 5.673 ± 0.112
		- rmse: 7.168 ± 0.121
		- r2_score: -1.509 ± 0.050
